In [1]:
!pip install azure-iot-device
!pip install azure-iot-hub
!pip install python-dotenv

  Using cached python_dotenv-0.14.0-py2.py3-none-any.whl (17 kB)


In [9]:
import sys
from dotenv import get_key, find_dotenv
sys.path.append('../utilities/video-analysis/shared/graph_manager')
from graph_manager import GraphManager

# find the environment file .env
env_path = find_dotenv(raise_error_if_not_found=True)

iotHubConnString = get_key(env_path, "IOT_HUB_CONN_STRING")
iotDeviceId = get_key(env_path, "IOT_DEVICE_ID")

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the "deployment.lvaedge_rocketserver_rtspsim.template.json" file
operationsApiVersion = "1.0"  # Must be same as the version number in the "topology.json" file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

### List existing topologies

In [11]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

{'additional_properties': {}, 'status': 200, 'payload': {'value': []}}


In [12]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

{'additional_properties': {}, 'status': 200, 'payload': {'value': []}}


In [20]:
topologyFile = os.path.abspath("../modules/Detector/httpExtension/topology.json")
operationParams = {
                    "topologyFile": topologyFile
                    }

In [21]:
response = graphManager.GraphTopologySet(operationParams)
print(response)

{'additional_properties': {}, 'status': 201, 'payload': {'systemData': {'createdAt': '2020-07-29T20:29:59.501Z', 'lastModifiedAt': '2020-07-29T20:29:59.501Z'}, 'name': 'InferencingWithHttpExtension', 'properties': {'description': 'Analyzing live video using HTTP Extension to send images to an external inference engine', 'parameters': [{'name': 'rtspUserName', 'type': 'String', 'description': 'rtsp source user name.', 'default': 'dummyUserName'}, {'name': 'rtspPassword', 'type': 'String', 'description': 'rtsp source password.', 'default': 'dummyPassword'}, {'name': 'rtspUrl', 'type': 'String', 'description': 'rtsp Url'}, {'name': 'inferencingUrl', 'type': 'String', 'description': 'inferencing Url', 'default': 'http://yolov3/score'}, {'name': 'inferencingUserName', 'type': 'String', 'description': 'inferencing endpoint user name.', 'default': 'dummyUserName'}, {'name': 'inferencingPassword', 'type': 'String', 'description': 'inferencing endpoint password.', 'default': 'dummyPassword'}, {

### Set actual topology parameters

In [30]:
rtspUrl = get_key(env_path, "CROSSING_VIDEO_URL")
inferencingUrl = get_key(env_path, "PEOPLE_DETECTOR")
topologyInstanceName =  "Sample-Graph-1"
topologyName = "InferencingWithHttpExtension"

mediaGraphTopologyParameters = {
          "name": topologyInstanceName,
          "properties": {
            "topologyName": topologyName,
            "description": "Sample graph description",
            "parameters": [
              {
                "name": "rtspUrl",
                "value": rtspUrl
              },
              {
                "name": "rtspUserName",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              },
              {
                "name": "inferencingUrl",
                "value": inferencingUrl
              },
              {
                "name": "inferencingUserName",
                "value": "username"
              },
              {
                "name": "inferencingPassword",
                "value": "password"
              },
              {
                "name": "imageEncoding",
                "value": "jpeg"
              },
              {
                "name": "imageScaleMode",
                "value": "pad"
              },
              {
                "name": "frameWidth",
                "value": "1920"
              },
              {
                "name": "frameHeight",
                "value": "1072"
              },
              {
                "name": "frameRate",
                "value": "30"
              }
            ]
          }
        }

#### Set topology instance

In [26]:
# Set topology instance
response = graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

{'additional_properties': {}, 'status': 201, 'payload': {'systemData': {'createdAt': '2020-07-29T20:42:29.103Z', 'lastModifiedAt': '2020-07-29T20:42:29.103Z'}, 'name': 'Sample-Graph-1', 'properties': {'state': 'Inactive', 'description': 'Sample graph description', 'topologyName': 'InferencingWithHttpExtension', 'parameters': [{'name': 'rtspUrl', 'value': 'rtsp://rtspsim/media/caffeteria.mkv'}, {'name': 'rtspUserName', 'value': 'username'}, {'name': 'rtspPassword', 'value': 'password'}, {'name': 'inferencingUrl', 'value': 'http://detector:5010/detect'}, {'name': 'inferencingUserName', 'value': 'username'}, {'name': 'inferencingPassword', 'value': 'password'}, {'name': 'imageEncoding', 'value': 'jpeg'}, {'name': 'imageScaleMode', 'value': 'pad'}, {'name': 'frameWidth', 'value': '1920'}, {'name': 'frameHeight', 'value': '1072'}, {'name': 'frameRate', 'value': '30'}]}}}


### Activate topology

In [29]:
# Activate topology instance
operationParams = {"name": topologyInstanceName}

response = graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

{'additional_properties': {}, 'status': 200, 'payload': None}


### Deactivate Graph

In [31]:
graphManager.GenericCall("GraphInstanceDeactivate", operationParams)
print(response)



{'additional_properties': {}, 'status': 200, 'payload': None}


### Delete Graph

In [32]:
graphManager.GenericCall("GraphInstanceDelete", operationParams)
print(response)

{'additional_properties': {}, 'status': 200, 'payload': None}


### Delete Topology

In [33]:
operationParams = {"name": topologyName}

graphManager.GenericCall("GraphTopologyDelete", operationParams)
print(response)

{'additional_properties': {}, 'status': 200, 'payload': None}
